In [34]:
from __future__ import print_function
%matplotlib inline

# Tutorial: understand how works FluidSim

## A class to organized parameters

In [35]:
from fluiddyn.util.paramcontainer import ParamContainer
params = ParamContainer(tag='params')
params._set_attribs({'a0': 1, 'a1': 1})
params._set_attrib('a2', 1)
params._set_child('child0', {'a0': 1})
params.a2 = 2
params.child0.a0 = 'other option'

A `ParamContainer` can be represented as xml

In [36]:
params


<params a1="1" a0="1" a2="2">
  <child0 a0="other option"/>  

</params>

FluidSim uses instances of this class to store the information of a solver and the parameters of a particular simulation.

## The Simul classes

The first step to run a simulation is to import a Simul class from a solver module, for example:

In [37]:
from fluidsim.solvers.ns2d.solver import Simul

Any solver module has to define a class called Simul which has to have some important attributes:

In [38]:
[name for name in dir(Simul) if not name.startswith('__')]

['InfoSolver',
 '_complete_params_with_default',
 'compute_freq_diss',
 'create_default_params',
 'info_solver',
 'tendencies_nonlin']

The first attribute `InfoSolver` is a class deriving from `ParamContainer`. It is used during the instantiation of the Simul object to produce a `ParamContainer` containing a description of the solver, in practice the classes used for the different tasks that need to be performed during the simulation.

There are also four other functions. `compute_freq_diss` and `tendencies_nonlin` are used during the simulation and describe the equations that are solved.

`create_default_params` and `_complete_params_with_default` are used to produce the `ParamContainer` containing the default parameters for a simulation:

In [39]:
params = Simul.create_default_params()

During the creation of `params`, the class `InfoSolver` has been used to create a `ParamContainer` named `info_solver`:

In [40]:
Simul.info_solver


<solver class_name="Simul" module_name="fluidsim.solvers.ns2d.solver"
        short_name="NS2D">
  <classes>
    <Operators class_name="OperatorsPseudoSpectral2D"
               module_name="fluidsim.operators.operators"/>  

    <InitFields class_name="InitFieldsNS2D"
                module_name="fluidsim.solvers.ns2d.init_fields">
      <classes>
        <from_file class_name="InitFieldsFromFile"
                   module_name="fluidsim.base.init_fields"/>  

        <noise class_name="InitFieldsNoise"
               module_name="fluidsim.solvers.ns2d.init_fields"/>  

        <constant class_name="InitFieldsConstant"
                  module_name="fluidsim.base.init_fields"/>  

        <jet class_name="InitFieldsJet"
             module_name="fluidsim.solvers.ns2d.init_fields"/>  

        <manual class_name="InitFieldsManual"
                module_name="fluidsim.base.init_fields"/>  

        <dipole class_name="InitFieldsDipole"
                module_name="fluidsim.solvers.ns2

We see that this solver uses many classes and that they are organized in tasks ("Operator", "InitFields", "TimeStepping", "State", "Output", "Forcing"). Some first-level classes (for example "Output") have second-level classes ("PrintStdOut", "Spectra", "PhysFields", etc.). Such description of a solver is very general. It is also very conveniant to create a new solver from a similar existing solver.

Every classes can have a class function `_complete_params_with_default` that is called when the object containing the default parameters is created.

The objects `params` and `Simul.info_solver` are then used to instantiate the simulation (here with the default parameters for the solver):

In [41]:
sim = Simul(params)

*************************************
Program FluidDyn

solver NS2D, RK4 and sequential,
type fft: FFTWCY
nx =     48 ; ny =     48
Lx = 8. ; Ly = 8.
path_run =
/home/users/augier3pi/Sim_data/NS2D_L=8.x8._48x48_2015-06-25_18-16-02
init_fields.type: constant
Initialization outputs:
<class 'fluidsim.base.output.increments.Increments'> increments
<class 'fluidsim.base.output.phys_fields.PhysFieldsBase'> phys_fields
<class 'fluidsim.solvers.ns2d.output.spectra.SpectraNS2D'> spectra
<class 'fluidsim.solvers.ns2d.output.spatial_means.SpatialMeansNS2D'> spatial_means
<class 'fluidsim.solvers.ns2d.output.spect_energy_budget.SpectralEnergyBudgetNS2D'> spect_energy_budg

Memory usage at the end of init. (equiv. seq.): 81.3125 Mo
Size of state_fft (equiv. seq.): 0.0192 Mo


Let's print the attributes of `sim` that are not class attributes:

In [42]:
[name for name in dir(sim) if not name.startswith('_') and name not in dir(Simul)]

['info',
 'init_fields',
 'name_run',
 'oper',
 'output',
 'params',
 'state',
 'time_stepping']

Except `name_run` and `info`, the attributes are instances of the first-level classes defined in `Simul.info_solver`. These different objects have to interact nicelly together. We are going to present these different hierarchies of classes but first we come back to the two functions describing the equations in a pseudo-spectral solver.

## Description of the solved equations

## State classes (`sim.state`)

## Operator classes (`sim.oper`)

## TimeStepping classes (`sim.time_stepping`)

## Output classes (`sim.output`)

## Forcing classes (`sim.forcing`)